In [ ]:
import os
import rasterio
from rasterio.windows import from_bounds
from rasterio.transform import from_bounds as transform_from_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling


In [ ]:
# Função para reprojetar o GeoTIFF para um CRS métrico (UTM)
def reproject_to_metric(input_path, output_path, target_crs):
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )

In [ ]:
# Caminhos
input_folder = r'C:\temp\drones\entrada'
output_base_folder = r'C:\temp\drones\saida'
temp_folder = r'C:\temp\drones\temp_reprojetado'
tile_size_meters = 1  # Tamanho do corte (1x1 metro)

os.makedirs(output_base_folder, exist_ok=True)
os.makedirs(temp_folder, exist_ok=True)

In [ ]:
# CRS de destino (exemplo: UTM Zona 23S para o Brasil, ajuste conforme necessário)
target_crs = 'EPSG:32723'

# Processar todos os arquivos .tif na pasta
for file_name in os.listdir(input_folder):
    if file_name.endswith('.tif'):
        input_geotiff = os.path.join(input_folder, file_name)
        reprojected_file = os.path.join(temp_folder, file_name)

        # Reprojetar para o CRS métrico
        reproject_to_metric(input_geotiff, reprojected_file, target_crs)

        # Diretório de saída para os cortes
        output_folder = os.path.join(output_base_folder, os.path.splitext(file_name)[0])
        os.makedirs(output_folder, exist_ok=True)

        with rasterio.open(reprojected_file) as src:
            bounds = src.bounds
            crs = src.crs
            transform = src.transform

            # Calcular passos de grade (em unidades do CRS projetado)
            x_min, y_min, x_max, y_max = bounds
            cols = int((x_max - x_min) / tile_size_meters)
            rows = int((y_max - y_min) / tile_size_meters)

            print(f"Processando {cols}x{rows} cortes para {file_name}...")

            # Gerar cortes
            for i in range(cols):
                for j in range(rows):
                    tile_min_x = x_min + i * tile_size_meters
                    tile_max_x = x_min + (i + 1) * tile_size_meters
                    tile_min_y = y_min + j * tile_size_meters
                    tile_max_y = y_min + (j + 1) * tile_size_meters

                    # Criar janela de corte
                    window = from_bounds(tile_min_x, tile_min_y, tile_max_x, tile_max_y, transform)
                    transform_tile = transform_from_bounds(tile_min_x, tile_min_y, tile_max_x, tile_max_y, window.width, window.height)

                    # Ler e salvar o corte
                    output_path = os.path.join(output_folder, f"tile_{i}_{j}.tif")
                    with rasterio.open(
                        output_path,
                        'w',
                        driver='GTiff',
                        height=window.height,
                        width=window.width,
                        count=src.count,
                        dtype=src.dtypes[0],
                        crs=crs,
                        transform=transform_tile,
                    ) as dst:
                        dst.write(src.read(window=window))

            print(f"Cortes salvos em: {output_folder}")
